# Лабораторная работа №1
## Выполнил студент группы БФИ2302 АндреевАМ

### Оглавление
1. [Задание 1](#Задание-№1)
2. [Задание 2](#Задание-№2)
3. [Задание 3](#Задание-№3)
4. [Вывод](#Вывод)

### Задание №1
<i> Вызвать функцию print() и передать туда строку Hello, World! </i>

In [15]:
print("Hello, World!")

Hello, World!


### Задание №2
Написать генератор случайных матриц(многомерных), который принимает
опциональные параметры <b>m</b>, <b>n</b>, <b>min_limit</b>, <b>max_limit</b>, где <b>m</b> и <b>n</b> указывают размер
матрицы, а <b>min_lim</b> и <b>max_lim</b> - минимальное и максимальное значение для
генерируемого числа.

In [ ]:
import random
import time
import numpy as np

In [ ]:
def userInput(prompt, default):
    u = input(prompt)
    return int(u) if u.strip() else default

def matrixGen():
    m = userInput("Rows: ",50)
    n = userInput("Columns: ",50)
    min = userInput("MinRand: ",-250)
    max = userInput("MaxRand: ",1000)
    nestedFor(m,n,min,max)
    listComp(m,n,min,max)
    numPy(m,n,min,max)

def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        func(*args, **kwargs)
        print("--- {0} ms ---".format(round((time.time() - start_time)*1000)))
    return wrapper

@timer
def nestedFor(m,n,min,max):
    matrix = []
    for row in range(m):
        a = []
        for col in range(n):
            a.append(random.randint(min,max))
        matrix.append(a)
    print(matrix)

@timer
def listComp(m,n,min,max):
    matrix = [[random.randint(min,max) for _ in range(n)] for _ in range(m)]
    print(matrix)

@timer
def numPy(m,n,min,max):
    matrix = np.random.randint(min,max+1,size=(m,n))
    print(matrix)

In [69]:
matrixGen()

[[7, 5, 2, 5, 8, 5], [6, 6, 7, 7, 1, 3], [8, 2, 6, 1, 7, 1], [2, 3, 3, 4, 3, 2], [4, 2, 9, 3, 7, 4], [2, 6, 5, 5, 4, 7]]
--- 0 ms ---
[[9, 2, 3, 4, 2, 5], [8, 4, 9, 2, 6, 7], [5, 3, 6, 3, 4, 3], [3, 3, 5, 1, 1, 3], [5, 6, 3, 6, 1, 5], [7, 2, 7, 1, 9, 6]]
--- 0 ms ---
[[9 5 4 2 7 2]
 [3 2 8 5 3 7]
 [6 3 9 1 9 2]
 [6 1 9 8 5 4]
 [9 3 9 8 8 5]
 [8 7 2 6 7 2]]
--- 0 ms ---


### Задание №3
Реализовать методы сортировки строк числовой матрицы в соответствии с
заданием. Оценить время работы каждого алгоритма сортировки и сравнить его со
временем стандартной функции сортировки. Испытания проводить на сгенерированных
матрицах.

In [25]:
# Сортировка выбором.
# Берётся срез массива, в котором минимальный элемент переносят в самый левый угол,
# после чего срез уменьшается и цикл повторяется.
start_time = time.time()
#TODO
print("--- {0} ms ---".format(round((time.time() - start_time)*1000)))

--- 0 ms ---


In [26]:
# Сортировка вставкой.
start_time = time.time()
#TODO
print("--- {0} ms ---".format(round((time.time() - start_time)*1000)))

--- 0 ms ---


In [27]:
# Сортировка обменом. || Сортировка пузырьком.
start_time = time.time()
#TODO
print("--- {0} ms ---".format(round((time.time() - start_time)*1000)))

--- 0 ms ---


In [28]:
# Сортировка Шелла.
start_time = time.time()
#TODO
print("--- {0} ms ---".format(round((time.time() - start_time)*1000)))

--- 0 ms ---


In [29]:
# Быстрая сортировка.
start_time = time.time()
#TODO
print("--- {0} ms ---".format(round((time.time() - start_time)*1000)))
    

--- 0 ms ---


In [30]:
# Турнирная сортировка.
start_time = time.time()
#TODO
print("--- {0} ms ---".format(round((time.time() - start_time)*1000)))

--- 0 ms ---


### Вывод
